# INFO 3350/6350

## Lecture 08: Fightin' words

## Measuring distinctive words between corpora

We often want to know which words are used differently in two corpora. There are a bunch of ways to do this. We can train classifiers and examine their feature weights. We can look at mutual information metrics. We could just count the words and see which ones are used more frequently in one corpus than another.

## Fightin' words

But a simple go-to approach that is robust to different underlying word frequencies and makes Bayesian assumptions about how often we would *expect* to see each word, given its frequency in a reference corpus, is Monroe et al.'s [Fightin' words](http://languagelog.ldc.upenn.edu/myl/Monroe.pdf).

Using code originally developed by Jack Hessel (a Cornell PhD grad!), we've provided you with `fightinwords.py`. We'll walk through that code and then see how it performs on real-world data.

The basic algorithm is to measure the observed frequency of each word in two corpora, (optionally) compare that frequency to an empirical prior, and normalize the result using a z-score. The words that have the largest magnitude z-scores (positive or negative) are the ones that tell us the most about the unique vocabulary of each corpus.

In [ ]:
import fightinwords as fw
import numpy as np
import os
from   sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# test on two novels with 40 more for informative priors

vectorizer = CountVectorizer( # set up a vectorizer
    lowercase=True,
    strip_accents='unicode',
    input='filename',
    encoding='utf-8',
)

# get file names
data_dir = os.path.join('..', 'data', 'texts')
files = os.listdir(data_dir)
bambi_file = 'O-Salten-Bambi-1923.txt'
mme_bovary_file = 'F-Flaubert-Madame_Bovary-1857-M.txt'
if '.DS_Store' in files: files.remove('.DS_Store') # remove MacOS cruft
files.remove(bambi_file)
files.remove(mme_bovary_file)
corpus = [os.path.join(data_dir, file) for file in files] # background corpus
samples = [os.path.join(data_dir, file) for file in [bambi_file, mme_bovary_file]]

# read target files by line
bambi_text = [fw.basic_sanitize(line) for line in open(samples[0], 'rt').readlines()]
mme_bovary_text = [fw.basic_sanitize(line) for line in open(samples[1], 'rt').readlines()]

# convenience function to FW display output
def display_fw(data, n=10, name1='corpus one', name2='corpus two'):
    '''Display the indicated number of top terms from fightinwords output.'''
    print("Top terms in", name1)
    for term, score in reversed(data[-n:]):
        print(f"{term:<10} {score:6.3f}")
    print("")
    print("Top terms in", name2)
    for term, score in data[:n]:
        print(f"{term:<10} {score:6.3f}")

In [ ]:
# results with a flat (noninformative) prior
# note idiom: pass in text, use default vectorizer
flat = fw.bayes_compare_language(bambi_text, mme_bovary_text)
display_fw(flat)

Meh. Not *wrong*, but I can't make much of this. Let's try it with an informative prior ...

In [ ]:
# learn vocab from *corpus* (not samples) and calculate priors
priors = np.sum(vectorizer.fit_transform(corpus), axis=0).reshape(-1,1)
priors.shape

Note that we have changed the input features in this case. We have learned word frequencies on a corpus the DOES NOT include either _Bambi_ or _Mme Bovary_. So we probably won't see "Bambi" as a feature.

In [ ]:
# vectorize test books using fitted vectorizer
test_books = vectorizer.transform(samples) # NOT .fit(); want to keep existing vocabulary from priors
print(test_books.shape)

In [ ]:
# use informative prior
# different idiom: pass in index positions in pre-computed feature matrix
informative = fw.bayes_compare_language(
    l1=[0], 
    l2=[1], 
    features=test_books, 
    cv=vectorizer, 
    prior=priors, 
    #prior_weight=10 <- optional normalize and reweight the data
)
display_fw(informative, name1='Bambi', name2='Mme Bovary')

## Plotting results

In [ ]:
from   adjustText import adjust_text # pretty, but not included with class environment
                                     # conda install -c conda-forge adjusttext 
import matplotlib.pyplot as plt
import pandas as pd

# convert to dataframe for convenience
df = pd.DataFrame(test_books.toarray(), columns=vectorizer.get_feature_names_out())

num_words_to_plot = 20 # number of most distinctive words to plot from each corpus

 # manage data for plotting
frequencies = []
zscores = []
words = []
for word, z_score in informative:
    count = df[word].sum()
    if count > 0:
        zscores.append(z_score)
        words.append(word)
        frequencies.append(count)

# plot result
texts = []
fig, ax = plt.subplots(1,1)
ax.scatter(
    np.log10(frequencies[:num_words_to_plot]), 
    zscores[:num_words_to_plot], 
    alpha=0.8, 
    label="Mme Bovary"
)
ax.scatter(
    np.log10(frequencies[-num_words_to_plot:]), 
    zscores[-num_words_to_plot:], 
    alpha=0.8, 
    label="Bambi"
)
for i in range(-num_words_to_plot, num_words_to_plot):
    texts.append(ax.text(np.log10(frequencies[i]), zscores[i], words[i], size='small', alpha=0.6))
adjust_text(texts, arrowprops=dict(arrowstyle="-", color='k', lw=0.5))
plt.xlabel('log(frequency)')
plt.ylabel('z-score')
plt.legend()
plt.tight_layout()
plt.show()

## News data

In [ ]:
# read data from disk and examine
import re

news = pd.read_csv(os.path.join('..', 'data', 'news', 'news_text.csv.gz'))

# a function to get rid of datelines at the start of articles
#  matches one or more hyphens or colons in first 40 chars,
#  drops everything before that match (plus the match itself)
pattern = '[-:]+ '
matcher = re.compile(pattern) # compiled regexs are faster

def remove_dateline(text, matcher=matcher):
    '''
    Remove source names and datelines from a text string
    If there is a hyphen or colon in the first 40 characters, 
      drops everything before the hyphen(s)/colon(s)
    If no hyphen/colon, do nothing
    Return processed string
    '''
    result = matcher.search(text, endpos=40)
    if result:
        return text[result.end():]
    else:
        return text

# clean article text
news['body'] = news['body'].apply(remove_dateline)

In [ ]:
num_holdout_articles = 20000

vec = CountVectorizer(
    lowercase=True,
    strip_accents='unicode',
    input='content',
    encoding='utf-8',
)

# calculate priors on non-holdout volumes
priors = np.sum(vec.fit_transform(news.body.iloc[num_holdout_articles:]), axis=0).reshape(-1,1)
priors.shape

In [ ]:
sports = news.iloc[:num_holdout_articles].loc[news.label=='Sports', ['body']]
other = news.iloc[:num_holdout_articles].loc[~(news.label=='Sports'), ['body']]
result = fw.bayes_compare_language(
    l1=[j for i, j in sports.itertuples()], 
    l2=[j for i, j in other.itertuples()], 
    cv=vec, 
    prior=priors
)

In [ ]:
display_fw(result, n=10, name1='sports', name2='other')

In [ ]:
print(f"Words in prior: {np.sum(priors):>10}")
print(f"Words in samples: {np.sum(vec.transform(news.body.iloc[:num_holdout_articles])):>8}")

In [ ]:
# what's up with '39'?
with pd.option_context("display.max_colwidth", 150):
    display(sports.loc[sports.body.str.contains('39')].sample(10))

In [ ]:
print("Fraction sports with '39':", len(sports.loc[sports.body.str.contains('39')])/len(sports))
print("Fraction other  with '39':", len(other.loc[other.body.str.contains('39')])/len(other))

Oh, well that's dumb. Clearly need more/different preprocessing. This is the sort of thing you want to check when you see odd results. That said, I guess sports articles use more apostrophes than do other articles?

In [ ]:
# compare without priors
no_priors = fw.bayes_compare_language(
    l1=[j for i, j in sports.itertuples()], 
    l2=[j for i, j in other.itertuples()], 
    cv=vec, 
)
display_fw(no_priors)